In [ ]:
def generate(prompt, drawer, iterations):
  torch.cuda.empty_cache()
  gc.collect()
  !rm -r output
  # prompt_input = "prompt_input_goes_here" # @param
  # project_name = "project_name_goes_here" # @param
  pixray.reset_settings()
  pixray.add_settings(prompts=prompt)
  pixray.add_settings(iterations=int(iterations))
  # pixray.add_settings(display_every=1)
  pixray.add_settings(batches=1)
  pixray.add_settings(learning_rate_drops=[100])
  pixray.add_settings(optimiser="DiffGrad")
  pixray.add_settings(clip_models="RN50,ViT-B/16,ViT-B/32")
  pixray.add_settings(quality="best")
  pixray.add_settings(size=[810,540])
  pixray.add_settings(display_clear="True")
  pixray.add_settings(make_video=False)
  pixray.add_settings(outdir="output")
  pixray.add_settings(drawer=drawer)

  settings = pixray.apply_settings()
  pixray.do_init(settings)
  pixray.do_run(settings)

  return 'output/output.png'

def launch_gradio():
  prompt = gr.inputs.Textbox(label="Image Prompt")
  drawer = gr.inputs.Radio(choices=["vqgan", "pixel", "clipdraw"], label="drawer")
  iterations = gr.inputs.Slider(10, 500, 10, 480, "Iterations")
  iface = gr.Interface(generate, inputs=[prompt, drawer, iterations], outputs=['image'], enable_queue=True, live=False)
  iface.launch(debug=True, share=True)

In [ ]:
!git clone --recursive --branch release https://github.com/pixray/pixray >nul 2>&1
!pip install -r pixray/requirements.txt >nul 2>&1
!pip install basicsr >nul 2>&1
!pip uninstall -y tensorflow >nul 2>&1
!pip install gradio >nul 2>&1
!git clone https://github.com/pixray/diffvg >nul 2>&1
%cd diffvg
!git submodule update --init --recursive >nul 2>&1
!python setup.py install >nul 2>&1
%cd ..
!pip freeze | grep torch >nul 2>&1

printsec = "printSecret"

if os.path.isfile("initiated"):
  import sys
  sys.path.append("pixray")
  import pixray
  import gc
  !mkdir models
  import gradio as gr
  import os
  launch_gradio();
else:
  !touch initiated >nul 2>&1
  print("initiated. ready to restart. " + printsec)